In [6]:
graph_path = "medical"
graph_oid = 96715

label_where_key = {
    "disease":"name",
    "drug":"name",
    "food":"name",
    "inspection_means":"name",
    "drug":"name",
    "recipe":"name",
    "manufacturer":"name",
    "second_level_departments":"name",
    "symptoms":"name",
    "treatment_plan":"name"
}


In [7]:
import torch
import torch_npu  
print(torch_npu.npu.is_available())

True


In [8]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import DirectoryLoader
model_name = "/home/work/liuytest/demo/maidalun1020/bce-embedding-base_v1"
model_kwargs = {'device': 'npu:0'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
 ## 将name打成向量
embeddings = model.embed_documents(
    [
        "entity_name"
    ]
)
embedding = embeddings[0]
embedding
print(len(embedding))
## pg_con.execute(f'INSERT INTO zyykg_data_{node} (embedding,label, value_name) VALUES (%s,%s,%s)', (embedding,label,entity_name))

768


In [9]:
## 将所有的ldbc schema下边的数据全部添加一列 设置为 embedding类型
import psycopg

pg_con = psycopg.connect(
            dbname="postgres",
            host="localhost",
            user="omm",
            password="ZAQ1234@esz",
            port=5432)
     
       
cur = pg_con.cursor()
pg_con.autocommit=True     
cur.execute(f"select labname from ag_label where graphid = {graph_oid} and labkind = 'v' and labname !='ag_vertex'");
rows = cur.fetchall()
for row in rows:
    labelname = str(row[0])
    if labelname in label_where_key:
        print(f"ALTER table {graph_path}.{row[0]} add COLUMN feature_emd vector(768)")
        cur.execute(f"ALTER table {graph_path}.{row[0]} add COLUMN feature_emd vector(768)")
pg_con.close()

ALTER table medical.second_level_departments add COLUMN feature_emd vector(768)
ALTER table medical.inspection_means add COLUMN feature_emd vector(768)
ALTER table medical.treatment_plan add COLUMN feature_emd vector(768)
ALTER table medical.manufacturer add COLUMN feature_emd vector(768)
ALTER table medical.disease add COLUMN feature_emd vector(768)
ALTER table medical.symptoms add COLUMN feature_emd vector(768)
ALTER table medical.drug add COLUMN feature_emd vector(768)
ALTER table medical.food add COLUMN feature_emd vector(768)
ALTER table medical.recipe add COLUMN feature_emd vector(768)


In [10]:
## 将所有的ldbc schema下边的数据全部添加一列 设置为 embedding类型

import psycopg
import json


 
pg_con = psycopg.connect(
            dbname="postgres",
            host="localhost",
            user="omm",
            password="ZAQ1234@esz",
            port=5432)
     
       
cur = pg_con.cursor()
pg_con.autocommit=True
   
cur.execute(f"select labname from ag_label where graphid = {graph_oid} and labkind = 'v' and labname !='ag_vertex'");
rows = cur.fetchall()
for row in rows:
    print('name=' + str(row[0]))
    labelname = str(row[0])
    if labelname in label_where_key:
        
        
        print(f"select * {graph_path}.{labelname} limit 1")
        cur.execute(f"select * from {graph_path}.{labelname}")

        # 获取列名
        column_names = [desc[0] for desc in cur.description]
        print(" | ".join(column_names))  # 打印列名

        # 获取查询结果
        results = cur.fetchall()

        where_key = label_where_key[labelname]

        # 打印结果
        for row in results:
            graphid = row[0] 
            properties = json.loads(json.dumps(row[1]))
            # if properties[where_key] is  not None:
            print(labelname)
            print(where_key)
            entity_value = properties[where_key]
            ## 将name打成向量
            embeddings = model.embed_documents(
                [
                    entity_value
                ]
            )
            embedding = embeddings[0]
            cur.execute(f"update   {graph_path}.{labelname}  set feature_emd = %s where id = %s",(embedding,graphid))
pg_con.close()

name=first_level_departments
name=second_level_departments
select * medical.second_level_departments limit 1
id | properties | feature_emd
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departments
name
second_level_departme